important libraries

In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

to check if your system can use gpu, if it prints cuda yeah the gpu is working

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [14]:
with open('Book.txt','r',encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)

['\n', '\x0c', ' ', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '©', '\xad', '·', '½', '×', 'à', '÷', '–', '—', '‘', '’', '“', '”', '•', '…', '€', '\uf02b', '\uf06e', '\uf071', '\uf092', '\uf094', '\uf0b4', '\uf0e6', '\uf0e7', '\uf0e8', '\uf0f6', '\uf0f7', '\uf0f8']


Tokenizers, we are using charcter level tokenizer, which it takes each character and converts into int. we are going to have very small vocabulary but so much tokens to convert.

In terms of LLM we are going to optimize the data by just not having a string of data, so we are going to use a framwork called pytorch (torch). which we going to use a data structure called tensors.

In [15]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s ]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data)

tensor([49, 50, 51,  ...,  0,  0,  1])


we are going to split it into train and validation splits, training 80% and validation 20%. To avoid memorization and overfitting.

we are going to use the bigram language model, lets take char "hello".
the bigram usally going to take like,
- start of content -> h
- h -> e
- e -> l
- l -> l
- l -> o

how are we going to use the bigram model into a Artificial neural network and train it. so we going to use block size. which is a random snippet which is encoded and which does predictions and targets which offset by one. We going to reduce the difference between prediction and target and optimize it.

block size = length of each sequence
batch size =  how many stack of sequence doin in th same time

we are going to be using nn.linear, it is important as nn module contains learnable paramters. when use weight or bias under nn module it learns it. when it trains it updates the weight or bias via backpropogation.

Embedding vecotor basically convert the character to a list of numbers, which is under nn module

@ - multiplying two matrices in torch or use matmul function

In pytorch, you cannot multiply int and float together

In [44]:
block_size = 8
batch_size = 4

n = int(0.8*len(data))

train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print('inputs:')
print(x)
print("targets:")
print(y)
    

tensor([ 178184,  821322, 1002337, 1366385])
inputs:
tensor([[81, 72, 80, 69, 82, 61, 80, 75],
        [61, 74, 64,  2, 82, 65, 68, 69],
        [74, 63, 75, 73, 65,  0, 69, 79],
        [69, 63, 68,  2, 63, 75, 73, 76]], device='cuda:0')
targets:
tensor([[81, 72, 80, 69, 82, 61, 80, 75, 78],
        [61, 74, 64,  2, 82, 65, 68, 69, 63],
        [74, 63, 75, 73, 65,  0, 69, 79,  2],
        [69, 63, 68,  2, 63, 75, 73, 76, 61]], device='cuda:0')
